In [4]:
from google.cloud import bigquery
from google.oauth2 import service_account
key_path = "./creds/service_acc_auth.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes = ["https://www.googleapis.com/auth/cloud-platform"])
bigquery_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [6]:
# query new data <= 1/1
union_weeks = """
SELECT * , 
FROM `scanning-database.scanning_data_backup.W50`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W51`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W52`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W53`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W01`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W02`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W03`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W04`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W05`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W06`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W07`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W08`
"""

In [8]:
# Overwriting old PowerBI source data
job_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10,
destination = 'scanning-database.PowerBI.PowerBI',
write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
) # set query limit to 10G
query_job = bigquery_client.query(union_weeks, job_config)  # API request
rows = query_job.result()  # Waits for query to finish
data_in_use = rows.to_dataframe()

In [9]:
data_in_use.sort_values(by=['Week']).tail()

,Start_Time,Finish_Time,Activity,Seq_Code,Recipe_Name,Break_Reasons,Missing_Ingredients,Kitting_Line,Assembly_Batch,Event_Shift,Team_Leader,Pickers_Count,Time_Consumption,Week
35426,2021-02-18 09:20:11+00:00,2021-02-18 09:24:10+00:00,Production,0020 of 0028,B2,None,None,KL2,Sun-SI Delivery,Thursday Morning,iXUsr_Gwen(7292),6,3.98,2021-08
35425,2021-02-18 10:14:22+00:00,2021-02-18 10:18:26+00:00,Production,0025 of 0029,B4,None,None,KL2,Sun-SI Delivery,Thursday Morning,iXUsr_Gwen(7292),6,4.07,2021-08
35424,2021-02-18 09:27:49+00:00,2021-02-18 09:32:11+00:00,Production,0022 of 0028,B2,None,None,KL2,Sun-SI Delivery,Thursday Morning,iXUsr_Gwen(7292),6,4.37,2021-08
35442,2021-02-18 07:10:41+00:00,2021-02-18 07:14:52+00:00,Production,0014 of 0029,B4,None,None,KL2,Sun-SI Delivery,Thursday Morning,iXUsr_Gwen(7292),6,4.18,2021-08
31552,2021-02-18 17:10:25+00:00,2021-02-18 17:11:01+00:00,Production,0009 of 0010,G4,None,None,KL4,Sat-NI Delivery,Thursday Afternoon,Melanie Dua;,4,0.60,2021-08


In [10]:
data_in_use.shape

(63105, 14)